# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
# Read in CSV
cities_df = pd.read_csv('output/city_data.csv')
cities_df.dropna(inplace = True)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nanortalik,60.1432,-45.2371,35.87,94,100,17.78,GL,1639882167
1,muros,42.7762,-9.0603,51.24,92,19,6.02,ES,1639882167
2,mataura,-46.1927,168.8643,74.68,44,91,12.48,NZ,1639882168
3,alugan,12.2188,125.4808,81.88,85,100,9.26,PH,1639882168
4,tamandare,-8.7597,-35.1047,77.85,79,70,8.28,BR,1639882168
...,...,...,...,...,...,...,...,...,...
544,gilbues,-9.8317,-45.3439,76.87,65,100,2.04,BR,1639882643
545,cockburn town,21.4612,-71.1419,79.83,65,1,22.84,TC,1639882644
546,ravar,31.2656,56.8055,55.76,36,85,4.92,IR,1639882644
547,murgab,37.4966,61.9714,53.47,37,90,6.91,TM,1639882644


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Set Locations Variable
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)


In [10]:
# Add Heatmap Layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity = 100, point_radius = 4)
fig.add_layer(heat_layer)

# Display Image
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [11]:
# Create New Dataframe for Weather Criteria
weather_df = cities_df.loc[(cities_df["Max Temp"] <= 75) & (cities_df["Max Temp"] >= 55) & (cities_df["Humidity"] <= 40) & (cities_df["Wind Speed"] >= 3)]
weather_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,abakaliki,6.3163,8.1169,72.82,33,62,4.94,NG,1639882252
102,tari,12.4086,4.4914,65.12,19,4,14.32,NG,1639882256
230,guerrero negro,27.9769,-114.0611,62.92,40,0,16.96,MX,1639882380
271,tera,14.0078,0.7531,66.43,18,4,10.63,NE,1639882421
295,khambhaliya,22.2072,69.6683,60.31,28,2,11.61,IN,1639882427
317,gao,16.6362,1.6370,61.97,18,1,13.78,ML,1639882463
321,rio gallegos,-51.6226,-69.2181,64.45,39,20,17.27,AR,1639882464
325,mirpur mathelo,28.0227,69.5489,57.06,15,6,4.83,PK,1639882465
365,taoudenni,22.6783,-3.9836,70.52,18,18,9.42,ML,1639882506
443,gashua,12.8710,11.0482,60.48,30,49,12.01,NG,1639882557


### Hotel Map

In [12]:
# Create hotel_df
hotel_df = weather_df[["City","Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

hotel_df

C:\Users\artsy\AppData\Local\Temp/ipykernel_152068/3310441060.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Country,Hotel Name
88,abakaliki,6.3163,8.1169,NG,
102,tari,12.4086,4.4914,NG,
230,guerrero negro,27.9769,-114.0611,MX,
271,tera,14.0078,0.7531,NE,
295,khambhaliya,22.2072,69.6683,IN,
317,gao,16.6362,1.6370,ML,
321,rio gallegos,-51.6226,-69.2181,AR,
325,mirpur mathelo,28.0227,69.5489,PK,
365,taoudenni,22.6783,-3.9836,ML,
443,gashua,12.8710,11.0482,NG,


In [13]:
# Setup Hotel Parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# For Loop
for index, hotel in hotel_df.iterrows():
    
    lat = hotel["Lat"]
    lng = hotel["Lng"]
    
    params["location"] = f"{lat}, {lng}"

    # Gmaps  Places API Call
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(url, params=params)
    
    hotel_info = response.json()
    
    # Use try to test the request
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    except:
        print("N/A skipping.")

hotel_df

C:\Users\artsy\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


N/A skipping.
N/A skipping.
N/A skipping.
N/A skipping.


,City,Lat,Lng,Country,Hotel Name
88,abakaliki,6.3163,8.1169,NG,Grande Vitesse Apartments and Hotels
102,tari,12.4086,4.4914,NG,
230,guerrero negro,27.9769,-114.0611,MX,"Casa Laguna, Bed & Breakfast"
271,tera,14.0078,0.7531,NE,Hotel Dar es salam
295,khambhaliya,22.2072,69.6683,IN,"Kalyan's Hotel, Banquet & Restaurant."
317,gao,16.6362,1.6370,ML,
321,rio gallegos,-51.6226,-69.2181,AR,Hotel Santa Cruz
325,mirpur mathelo,28.0227,69.5489,PK,Leopards courier mirpur mathelo
365,taoudenni,22.6783,-3.9836,ML,
443,gashua,12.8710,11.0482,NG,Forte Oil - Old Lorry Park


In [14]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))